In [2]:
from itertools import product
from tqdm import tqdm
import csv
from data_generation import word
import opencc
import os


2023-01-20 19:59:55.542154: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/yuan/anaconda3/envs/wtcpt/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/yuan/anaconda3/envs/wtcpt/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
2023-01-20 19:59:57.173345: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-20 19:59:5

(['傅达仁今运行安乐死，爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。', '傅达仁今运行安乐死，爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。'], ['0000100000100000000000000000010000000000', '0000000000000000000000000000010000000000'], [['傅达仁今MASK运行安乐死，却突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。', '傅达仁今将运行安乐死，却突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。'], ['傅达仁今将运行安乐死，MASK突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。', '傅达仁今将运行安乐死，却突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。'], ['傅达仁今将运行安乐死，却MASK爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。', '傅达仁今将运行安乐死，却突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。'], ['傅达仁今将运行安乐死，却突然爆出自己20年前遭纬来体育台封杀，他MASK懂自己哪里得罪到电视台。', '傅达仁今将运行安乐死，却突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。']])


In [3]:
sents_file = 'sents.csv'
sents_ = []
with open(sents_file,'r') as f:
    sents_ = [i for i in csv.reader(f)][1:]

sents = sents_[:2]

In [1]:
'test'[:10]

'test'

In [45]:
def get_len(a):
    return(len(a))

def word_data(sents:list, len:list, p:list, pos_tag:str):
    if not os.path.isdir('word'):
        os.mkdir('word')

    sents_ = []
    mask = []
    label = []

    for sent in sents:
        sent = sent[:max(len)]
        print(word(sent, pos_tag, p))
        a, b, c = word(sent, pos_tag, p)
        print(a,b,c)
        label.append([a,b])
        mask += c
    
    for r in range(get_len(p)):
        print(mask,'0',label)
        for y in len:
            mask_ = [[mask[i][0][r][:y] for i in range(get_len(label))],[mask[i][1][r][:y] for i in range(get_len(label))]]
            print(r,y,mask_)

word_data(['傅达仁今将运行安乐死，突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。'], [64,32], [0.5,0.8], 'D')

(['傅', '傅'], ['0', '0'], [])
['傅', '傅'] ['0', '0'] []
[] 0 [[['傅', '傅'], ['0', '0']]]


IndexError: list index out of range

In [11]:
class label_token():
    def __init__(self, sents, max, p, pos_tag):
        
        self.max_length = max
        self.p = p
        self.pos_tag = pos_tag

        self.result = []
        self.generate()

    def label_padding(self, label):
        while len(label) < self.max_length:
            label+=('0')
        return label

    def generate(self):
        
        if not os.path.isdir('tokenLabel'):
            os.mkdir('tokenLabel')

        for sent in tqdm(sents):
            
            try:
                if len(sent) > self.max_length:
                    sent = sent[0:self.max_length-1]

                s_text, s_label = word.label(sent,self.pos_tag,self.p)
                
                s_label = self.label_padding(s_label)

                if '1' in s_label:
                    self.result.append([s_text,s_label])
                    
            except:
                continue
        
        with open('tokenLabel/tokenLabel_'+str(len(self.result))+'_'+self.pos_tag+'_'+str(self.max_length)+'_'+str(self.p)+'_'+'.csv', 'w', newline='') as f:

            fieldnames = ['text']
            for i in range(self.max_length):
                fieldnames.append(str(i))

            writer = csv.writer(f)
            writer.writerow(fieldnames)

            for i in self.result:
                writer.writerow([i[0]]+list(i[1]))
            

In [13]:
class mask_token():
    def __init__(self, sents, pos_tag):
        
        self.pos_tag = pos_tag

        self.result = []
        self.generate()

    def generate(self):
        
        if not os.path.isdir('tokenMask'):
            os.mkdir('tokenMask')

        for sent in tqdm(sents):
            
            # try:
                res = word.mask(sent,self.pos_tag)
                for row in res:
                    text = row[0]
                    target = row[0].replace('MASK',row[1])
                    self.result.append([text, target])
                    
            # except:
            #     continue
        
        with open('tokenMask/tokenMask_'+str(len(self.result))+'_'+self.pos_tag+'.csv', 'w', newline='') as f:

            fieldnames = ['text', 'target']

            writer = csv.writer(f)
            writer.writerow(fieldnames)

            for i in self.result:
                writer.writerow(i)
            

In [16]:
lengths = [96]
ps = [0.8]
tags = ['A','D','P','Nf','Cbb']

In [12]:
for length,p,tag in product(lengths,ps,tags):
    print(length, p, tag)
    label_token(sents, length, p, tag)

96 0.8 A


  0%|          | 214/200000 [00:15<3:41:09, 15.06it/s]

In [17]:
for tag in tags:
    print(tag)
    mask_token(sents, tag)


A


100%|██████████| 20/20 [00:01<00:00, 12.17it/s]


D


100%|██████████| 20/20 [00:01<00:00, 15.48it/s]


P


100%|██████████| 20/20 [00:01<00:00, 14.53it/s]


Nf


100%|██████████| 20/20 [00:01<00:00, 15.78it/s]


Cbb


100%|██████████| 20/20 [00:01<00:00, 15.09it/s]


In [6]:
# lengths = [96]
# ps = [0.5,0.2,0.8]
# tags = ['A','D','P','Nf','Cbb']

# for length,p,tag in product(lengths,ps,tags):
#     print(length, p, tag)
#     label_token(sents, length, p, tag)